In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import requests
import os
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
email = os.getenv("GMAIL_TO")
print(email)

hiraakhann99@gmail.com


In [4]:
RESEND_API_KEY = os.getenv("RESEND_API_KEY")

Adding Instructions Prompts


In [ ]:
helpful_prompt_instructions = "You are a helpful assistant agent who will help me with breaking down any tasks \
    or problem into smaller steps and then compile them in an email."

funny_prompt_instructions = "You are a funny assistant agent who will share funny jokes and puns with me while \
    also helping me with my tasks."

In [ ]:
helpful_agent = Agent(
    role="Assistant",
    goal=helpful_prompt_instructions,
    model="gpt-4o-mini",
)

funny_agent = Agent(
    role="Assistant",
    goal=funny_prompt_instructions,
    model="gpt-4o-mini",
)


In [ ]:
tool_description = "write the email to the user"

helpful_tool = helpful_agent.as_tool(tool_description)
funny_tool = funny_agent.as_tool(tool_description)

In [ ]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all users via Resend """
    
    # Set up email sender, recipient, and content
    from_email = "hira@resend.dev" # Replace with your verified sender
    to_email = os.getenv("GMAIL_TO")  # Replace with recipient's email
    
    # Resend API headers and payload
    headers = {
        "Authorization": f"Bearer {RESEND_API_KEY}",
        "Content-Type": "application/json"
    }
    
    payload = {
        "from": f"Hira Khan :p <{from_email}>",
        "to": [to_email],
        "subject": "Funny email",
        "html": f"<p>{body}</p>"  # Body wrapped in <p> tags for HTML format
    }
    
    # Send email using Resend API
    response = requests.post("https://api.resend.com/emails", json=payload, headers=headers)
    
    # Check if the request was successful
    if response.status_code == 202:
        return {"status": "success"}
    else:
        return {"status": "failure", "message": response.text}

In [ ]:
subject_instructions = "You can write a subject for a funny email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")

In [ ]:
email_tools = [subject_tool, html_tool, send_email]

In [ ]:
instructions ="You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the send_html_email tool to send the email with the subject and HTML body."


emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=email_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")

In [ ]:
tools = [helpful_tool, funny_tool]
handoffs = [emailer_agent]

In [ ]:
manager_instructions = """
You are a my personal manager. Your goal is to find the single best email using the agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all two agent tools to generate 2 different email drafts. Do not proceed until all 2 drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
You can use the tools multiple times if you're not satisfied with the results from the first try.
 
3. Handoff for Sending: Pass ONLY the winning email draft to the 'Email Manager' agent. The Email Manager will take care of formatting and sending.
 
Crucial Rules:
- You must use the agent tools to generate the drafts — do not write them yourself.
- You must hand off exactly ONE email to the Email Manager — never more than one.
"""


sales_manager = Agent(
    name="Manager",
    instructions=manager_instructions,
    tools=tools,
    handoffs=handoffs,
    model="gpt-4o-mini")

message = "Send out an email addressed to Dear Test from Hira"

with trace("Hira's Manager"):
    result = await Runner.run(sales_manager, message)

In [ ]:
class NameCheckOutput(BaseModel):
    is_name_in_message: bool
    name: str

guardrail_agent = Agent( 
    name="Name check",
    instructions="Check if the user is including someone's personal name in what they want you to do.",
    output_type=NameCheckOutput,
    model="gpt-4o-mini"
)

In [ ]:
@input_guardrail  # pyright: ignore[reportUndefinedVariable]
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx.context)
    is_name_in_message = result.final_output.is_name_in_message
    return GuardrailFunctionOutput(output_info={"found_name": result.final_output},tripwire_triggered=is_name_in_message)  # pyright: ignore[reportUndefinedVariable]

In [ ]:
careful_sales_manager = Agent(
    name="Manager",
    instructions=manager_instructions,
    tools=tools,
    handoffs=[emailer_agent],
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
    )

message = "Send out a email addressed to Dear Test from Hira"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)

with trace("Protected Hira's Manager"):
    result = await Runner.run(sales_manager, message)

In [ ]:
message = "Send out email addressed to Dear CEO from Head of Business Development"

with trace("Protected Automated SDR"):
    result = await Runner.run(careful_sales_manager, message)